In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import joblib

df = pd.read_csv("../data/processed/feature_engineered_holdout.csv")
df['date'] = pd.to_datetime(df['date'])

print("Training Forecasting Model...")

# 1. Aggregate Data by Month
# We predict the Market Average, not individual houses
monthly_data = df.set_index('date').resample('M')['price'].mean()

# Fill gaps if any months have no sales
monthly_data = monthly_data.interpolate(method='linear')

# 2. Train Holt-Winters Model
# Trend='add' (linear trend), Seasonal='add' (recurring pattern)
model = ExponentialSmoothing(
    monthly_data, 
    trend='add', 
    seasonal='add', 
    seasonal_periods=12
).fit()

joblib.dump(model, "../models/forecasting.joblib")
print("✅ Forecasting Model Saved")